In [1]:
##############################################################################
#####################################################################
#####################################################################
#
# OLS: 
#     all columns ⇔ sold-houses prices
#
#####################################################################
##############################################################################

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp


In [3]:
# import clean data (cleaned in step 1)
df = pd.read_csv("../data/cleandata.csv")
# confirm
df.head(1)


,Unnamed: 0,const,id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,month,year,SaleType,SaleCondition,price,date
0,1404,1.0,1405,50,RL,60.0,10410,1,Reg,Lvl,...,0,0,0,0,1,2006,WD,Family,105000,2006-01-01


In [6]:
# savety copy
wdf = df.copy()

In [40]:
# get corr df from step 2
corrdf = pd.read_csv('corrdf.csv')
# confirm
corrdf = corrdf.sort_values('abs_price_corr',ascending=False)


In [ ]:
# get top10 cols + price

# get top10 cols
top10 = corrdf['col'].to_list()[0:11]

In [24]:
# svate to df
top10df = wdf.copy()
top10df = top10df.get(top10)

In [30]:
# add constant
top10df = sm.add_constant(top10df)

In [31]:
#review
top10df

,const,price,OverallQual,GrLivArea,GarageCars,GarageArea,1stFlrSF,TotalBsmtSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd
0,1.0,105000,3,1214,3,936,694,672,1,6,1915,1950
1,1.0,136500,5,1050,1,368,1050,1050,1,6,1961,1961
2,1.0,145000,5,1056,2,572,1056,1056,1,5,1955,1955
3,1.0,155000,5,1803,2,484,1803,1134,2,8,1957,2000
4,1.0,172400,6,1664,2,460,953,953,2,7,2000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,1.0,80000,4,924,1,280,672,672,1,5,1941,1950
1334,1.0,127500,6,1069,2,440,1069,1069,2,4,1977,1977
1335,1.0,142000,5,1056,1,304,1056,1056,1,6,1968,1968
1336,1.0,146000,6,1456,2,440,855,855,2,7,1978,1978


In [32]:
top10df.columns

Index(['const', 'price', 'OverallQual', 'GrLivArea', 'GarageCars',
       'GarageArea', '1stFlrSF', 'TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd',
       'YearBuilt', 'YearRemodAdd'],
      dtype='object')

In [34]:
# OLS: all columns

    # ols
results = sm.OLS(top10df["price"], 
                  top10df[['const','OverallQual', 'GrLivArea', 'GarageCars','GarageArea', '1stFlrSF', 'TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd','YearBuilt', 'YearRemodAdd']]).fit()
    # summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     434.0
Date:                Sun, 13 Sep 2020   Prob (F-statistic):               0.00
Time:                        18:36:48   Log-Likelihood:                -16014.
No. Observations:                1338   AIC:                         3.205e+04
Df Residuals:                    1327   BIC:                         3.211e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.191e+06    1.4e+05     -8.517      0.000   -1.47e+06   -9.17e+05
OverallQual    2.09e+04   1282.432     16.298      0.000    1.84e+04    2.34e+04
GrLivArea       51.0923      4.445     11.495      0.000      42.373      59.812
GarageCars    1.267e+04   3288.510      3.852      0.000    6217.717    1.91e+04
GarageArea      16.2097     10.624      1.526      0.127      -4.633      37.052
1stFlrSF        11.1226      6.652      1.672      0.095      -1.926      24.171
TotalBsmtSF     22.3432      6.290      3.552      0.000      10.003      34.683
FullBath     -9688.2671   2887.891     -3.355      0.001   -1.54e+04   -4022.937
TotRmsAbvGrd  1070.2700   1203.571      0.889      0.374   -1290.838    3431.378
YearBuilt      246.1478     55.877      4.405      0.000     136.531     355.764
YearRemodAdd   313.3636     70.442      4.449      0.000     175.174     451.553
==============================================================================
Omnibus:                      515.944   Durbin-Watson:                   1.612
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            63287.225
Skew:                          -0.745   Prob(JB):                         0.00
Kurtosis:                      36.660   Cond. No.                     4.85e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.85e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [36]:
# OLS: Top1 OverallQual
    # ols
results = sm.OLS(top10df["price"], 
                  top10df[['const','OverallQual']]).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     2125.
Date:                Sun, 13 Sep 2020   Prob (F-statistic):          2.12e-278
Time:                        18:40:23   Log-Likelihood:                -16349.
No. Observations:                1338   AIC:                         3.270e+04
Df Residuals:                    1336   BIC:                         3.271e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.036e+05   6440.614    -16.086      0.000   -1.16e+05    -9.1e+04
OverallQual  4.668e+04   1012.821     46.094      0.000    4.47e+04    4.87e+04
==============================================================================
Omnibus:                      543.399   Durbin-Watson:                   1.526
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5291.081
Skew:                           1.612   Prob(JB):                         0.00
Kurtosis:                      12.193   Cond. No.                         31.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
# OLS: Top2 GrLivArea 

    # ols
results = sm.OLS(top10df["price"], 
                  top10df[['const','GrLivArea']]).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     1371.
Date:                Sun, 13 Sep 2020   Prob (F-statistic):          3.86e-207
Time:                        18:41:43   Log-Likelihood:                -16513.
No. Observations:                1338   AIC:                         3.303e+04
Df Residuals:                    1336   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.088e+04   4729.145      4.415      0.000    1.16e+04    3.02e+04
GrLivArea    107.8863      2.913     37.032      0.000     102.171     113.602
==============================================================================
Omnibus:                      247.036   Durbin-Watson:                   1.488
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3901.300
Skew:                           0.357   Prob(JB):                         0.00
Kurtosis:                      11.335   Cond. No.                     5.06e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
# OLS: Top3 GarageCars

    # ols
results = sm.OLS(top10df["price"], 
                  top10df[['const','GarageCars']]).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.410
Model:                            OLS   Adj. R-squared:                  0.409
Method:                 Least Squares   F-statistic:                     927.6
Date:                Sun, 13 Sep 2020   Prob (F-statistic):          3.64e-155
Time:                        18:42:13   Log-Likelihood:                -16633.
No. Observations:                1338   AIC:                         3.327e+04
Df Residuals:                    1336   BIC:                         3.328e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.771e+04   5167.057      7.298      0.000    2.76e+04    4.78e+04
GarageCars  7.964e+04   2614.988     30.457      0.000    7.45e+04    8.48e+04
==============================================================================
Omnibus:                      513.035   Durbin-Watson:                   1.439
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4586.140
Skew:                           1.528   Prob(JB):                         0.00
Kurtosis:                      11.540   Cond. No.                         7.60
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
# cross-checking with month

# get month and assign to df
top10df['month'] = wdf['month']

In [45]:
# OLS: crosscheck month

    # ols
results = sm.OLS(top10df["price"], 
                  top10df[['const','month']]).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.284
Date:                Sun, 13 Sep 2020   Prob (F-statistic):              0.131
Time:                        18:45:35   Log-Likelihood:                -16984.
No. Observations:                1338   AIC:                         3.397e+04
Df Residuals:                    1336   BIC:                         3.398e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.791e+05   5500.151     32.566      0.000    1.68e+05     1.9e+05
month       1207.6223    799.109      1.511      0.131    -360.023    2775.267
==============================================================================
Omnibus:                      587.270   Durbin-Watson:                   0.849
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3546.498
Skew:                           1.957   Prob(JB):                         0.00
Kurtosis:                       9.949   Cond. No.                         17.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
##################################################################################
##################################################################################
### NOTES:
#
#       R-squared:
#                  - month:        0.002
#
#                  - GarageCars:   0.410
#                  - GrLivArea:    0.507
#                  - OverallQual:  0.614
#
#
##################################################################################
##################################################################################
